In [ ]:
import math
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()
from dense_correspondence.training.training import *
import sys
import logging
import copy

# utils.set_default_cuda_visible_devices()
utils.set_cuda_visible_devices([0]) # use this to manually set CUDA_VISIBLE_DEVICES

from dense_correspondence.training.training import DenseCorrespondenceTraining
from dense_correspondence.dataset.spartan_dataset_masked import SpartanDataset
logging.basicConfig(level=logging.INFO)


In [ ]:
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'dataset', 'composite', 'caterpillar_only.yaml')
config = utils.getDictFromYamlFilename(config_filename)

train_config_file = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'training', 'caterpillar_training.yaml')

train_config = utils.getDictFromYamlFilename(train_config_file)
train_config_original = copy.deepcopy(train_config)

dataset = SpartanDataset(config=config)
logging_dir = "code/data_volume/pdc/trained_models/lucas"
num_iterations = 3500

In [ ]:
descriptor_dim = [3]
network_dict = dict()
d = 3
exponent_list = range(-4,4)
for exponent in exponent_list:
    non_match_loss_weight = math.pow(10, exponent)
    print "training descriptor of dimension %d" %(d)
    name = "multi_object_weight_e%s_%sd" %(exponent, d)
    print non_match_loss_weight
    print name
    
    train_config = copy.deepcopy(train_config_original)
    
    train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
    train._config["training"]["logging_dir"] = logging_dir
    train._config["training"]["logging_dir_name"] = name
    train._config["dense_correspondence_network"]["descriptor_dimension"] = d
    train._config["training"]["num_iterations"] = num_iterations
    train._config["scale_by_hard_negatives"] = False
    train._config["training"]["domain_randomize"] = False
    train.run()
    print "finished training descriptor of dimension %d" %(d)
    
    
    print "running evaluation"
    model_folder = os.path.join(logging_dir, name)
    DCE.run_evaluation_on_network(model_folder, num_image_pairs=25)
    